Part 3

In [24]:
# import the relevant libraries
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import string
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [25]:
#read data file
tweets = pd.read_csv('data.csv' ,encoding="utf-8")
len(tweets)

513

In [26]:
#Emoji sentiment database
emojisentiment = pd.read_excel('emojisentiment.xlsx')
emojisentiment = pd.read_excel('emojisentiment.xlsx',usecols=['emoji', 'sentiment'])
print(emojisentiment.to_dict(orient='record'))

[{'emoji': '\\xF0\\x9F\\x98\\x82', 'sentiment': 0.221}, {'emoji': '\\xF0\\x9F\\x98\\x8D', 'sentiment': 0.678}, {'emoji': '\\xF0\\x9F\\x98\\x8A', 'sentiment': 0.644}, {'emoji': '\\xF0\\x9F\\x98\\xAD', 'sentiment': -0.093}, {'emoji': '\\xF0\\x9F\\x98\\xA9', 'sentiment': -0.368}, {'emoji': '\\xF0\\x9F\\x98\\x92', 'sentiment': -0.374}, {'emoji': '\\xF0\\x9F\\x98\\xB3', 'sentiment': 0.018}, {'emoji': '\\xF0\\x9F\\x98\\xA1', 'sentiment': -0.173}, {'emoji': '\\xF0\\x9F\\x98\\xB1', 'sentiment': 0.19}, {'emoji': 'dY~,', 'sentiment': 0.221}, {'emoji': 'dY~_', 'sentiment': 0.678}, {'emoji': 'dY~T', 'sentiment': 0.644}, {'emoji': 'dY~-', 'sentiment': -0.093}, {'emoji': 'dY~c', 'sentiment': -0.368}, {'emoji': "dY~'", 'sentiment': -0.374}, {'emoji': 'dY~3', 'sentiment': 0.018}, {'emoji': 'dY~\xad', 'sentiment': -0.173}, {'emoji': 'dY~ñ', 'sentiment': 0.19}, {'emoji': ':face_with_tears_of_joy:', 'sentiment': 0.221}, {'emoji': ':smiling_face_with_heart-eyes:', 'sentiment': 0.678}, {'emoji': ':smiling_

In [27]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
#Finding emoji 
emojisentiment = dict(emojisentiment.values.tolist())  #Converting to dicts
i = 0
all_tweets = list(tweets['text'])
emoji = [] #empty series to hold our predicted values
for tweet in all_tweets:
  nltk_tokens = nltk.word_tokenize(str(tweet))
  print(nltk_tokens)
  for token in nltk_tokens:
    if token in emojisentiment.keys():
      print(token)
      e= token
  emoji.append(e)

tweets['emoji'] = emoji 

['.', 'I', 'really', 'love', 'the', 'song', '.', 'So', 'relatable', '@', 'ALIEGATIE', 'LOVE', '\\xF0\\x9F\\x98\\x82', '\\xF0\\x9F\\x98\\x8D']
\xF0\x9F\x98\x82
\xF0\x9F\x98\x8D
['People', 'should', 'start', 'watching', 'the', 'not', 'mainstream', 'movies', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['I', 'am', 'so', 'happy', 'I', 'am', 'able', 'to', 'bring', 'phone', 'to', 'my', 'school.\\xF0\\x9F\\x98\\x82', '@', 'SHARKISes_', 'are', "n't", 'we', 'happy', 'about', 'it', '?', '?', '?', '?', '\\xF0\\x9F\\x98\\xB3']
\xF0\x9F\x98\xB3
['The', 'actors', 'in', '#', 'PeakyBlinders', 'are', 'too', 'good', '.', 'I', 'am', 'excited', 'to', 'the', 'next', 'season', '#', 'excited', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['The', 'way', '@', 'RyanMcMullan', '@', 'ryavanhhuu__', 'sings', 'are', 'so', 'heavenly', '.', 'People', 'listen', 'to', 'him', '#', 'waytooggo', '\\xF0\\x9F\\x98\\x8A']
\xF0\x9F\x98\x8A
['CANT', 'WAIT', 'To', 'WATCH', '@', 'RIVERDALE', 'NEW', 'SEASON', '!', '\\xF0\\x9F\\x98\\x8A',

In [29]:
# data preprocessing (pt1)
#function to work with Regex
def remove_pattern(input_text, pattern):
    r = re.findall(pattern, input_text)
    for i in r:
        input_text = re.sub(i, '', input_text)
    
    return input_text

# removing twitter handles (e.g @username)
tweets['text'] = np.vectorize(remove_pattern)((tweets['text']), "@[\w]*")


# removing hastags
tweets['text'] = np.vectorize(remove_pattern)(tweets['text'], "#[\w]*")


# remove blank rows if any
tweets['text'].dropna(inplace = True)


# change text to lowercase
tweets['text'] = [entry.lower() for entry in tweets['text']]


# tokenizing the text
tweets['text'] = [word_tokenize(entry) for entry in tweets['text']]

In [30]:
import nltk
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
# data preprocessing (pt2)
# removing Stop words, Non-Numeric and perfom Word Stemming/Lemmatization
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(tweets['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    tweets.loc[index,'text_final'] = str(Final_words)

In [32]:
tweets['final'] = (tweets[['text_final','emoji']].agg(','.join, axis=1))  
tweets.head()

,tweet_id,text,emoji,text_final,final
0,1,"[., i, really, love, the, song, ., so, relatab...",\xF0\x9F\x98\x8D,"['really', 'love', 'song', 'relatable', 'love']","['really', 'love', 'song', 'relatable', 'love'..."
1,2,"[people, should, start, watching, the, not, ma...",\xF0\x9F\x98\x8A,"['people', 'start', 'watch', 'mainstream', 'mo...","['people', 'start', 'watch', 'mainstream', 'mo..."
2,3,"[i, am, so, happy, i, am, able, to, bring, pho...",\xF0\x9F\x98\xB3,"['happy', 'able', 'bring', 'phone', 'happy']","['happy', 'able', 'bring', 'phone', 'happy'],\..."
3,4,"[the, actors, in, are, too, good, ., i, am, ex...",\xF0\x9F\x98\x8A,"['actor', 'good', 'excite', 'next', 'season']","['actor', 'good', 'excite', 'next', 'season'],..."
4,5,"[the, way, sings, are, so, heavenly, ., people...",\xF0\x9F\x98\x8A,"['way', 'sings', 'heavenly', 'people', 'listen']","['way', 'sings', 'heavenly', 'people', 'listen..."


In [33]:
#Load the saved Naive Bayes Model
import pickle
f = open('naivebayesclassifier.pickle', 'rb')
Naive = pickle.load(f)
f.close()

In [34]:
#Predict sentiment for whole dataset
from sklearn.feature_extraction.text import TfidfVectorizer
Tfidf_vect = TfidfVectorizer(max_features=5000)
_tweet = Tfidf_vect.fit_transform(tweets['final'])
predict = Naive.predict(_tweet)
print( predict)
print(tweets['final'][16])

[2 2 2 2 2 2 2 2 2 0 0 1 0 0 2 0 1 0 0 0 2 2 2 2 2 2 2 1 2 2 0 0 0 0 2 0 2
 2 0 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 0 0 0 1 0 2 0 0 1 2 2 0 0 1
 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 0 0 0 1 1 0 2 0 0 1 0 0 0 2 2 0 0 0 0 0 1 1 2 1 0 0
 2 2 0 0 0 0 2 2 0 2 2 2 1 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 1 0 0 0
 2 0 0 2 0 0 1 0 0 2 1 2 2 2 0 1 2 2 2 2 2 0 0 2 0 0 0 0 0 0 0 1 1 0 1 1 2
 2 2 2 2 2 2 2 2 2 0 0 1 1 0 2 0 0 2 1 1 0 1 0 1 1 2 1 0 1 0 1 0 2 2 2 2 0
 0 2 2 0 1 0 0 2 0 2 2 0 0 2 2 0 0 0 0 0 0 0 2 2 0 1 0 0 0 1 1 1 1 1 1 2 0
 2 0 1 0 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 2
 1 1 1 2 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 1 1 1 1 1 1 2 1
 1 1 1 2 1 1 2 1 2 2 1 1 1 2 1 1 1 1 2 1 1 2 0 2 1 1 1 1 2 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 2 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 2 1 1 1 0 1 0 0 1 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 

In [35]:
#Assigning sentiment for each texts in data
data = pd.read_csv('data.csv' ,encoding="utf-8")
i=0
sent = [] #empty series to hold sentiment
while(i<len(data)):
  for i in range(0, len(predict)):
    sentiment = (predict[i])
    sent.append(sentiment)
    i = i+1
data['sentiment'] = sent
data.head()

,tweet_id,text,sentiment
0,1,. I really love the song. So relatable @ALIEGA...,2
1,2,People should start watching the not mainstrea...,2
2,3,I am so happy I am able to bring phone to my s...,2
3,4,The actors in#PeakyBlinders are too good. I am...,2
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,2


In [36]:
#Assigning score categories and logic
i = 0

predicted_value = [ ] #empty series to hold our predicted values

while(i<len(data)):
    if ((data.iloc[i]['sentiment'] == 2)):
        predicted_value.append('positive')
        i = i+1
    elif ((data.iloc[i]['sentiment'] == 1)):
        predicted_value.append('neutral')
        i = i+1
    elif ((data.iloc[i]['sentiment'] == 0)):
        predicted_value.append('negative')
        i = i+1
data['predicted sentiment'] = predicted_value       
data.head(20)

,tweet_id,text,sentiment,predicted sentiment
0,1,. I really love the song. So relatable @ALIEGA...,2,positive
1,2,People should start watching the not mainstrea...,2,positive
2,3,I am so happy I am able to bring phone to my s...,2,positive
3,4,The actors in#PeakyBlinders are too good. I am...,2,positive
4,5,The way @RyanMcMullan @ryavanhhuu__ sings are ...,2,positive
5,6,CANT WAIT To WATCH @RIVERDALE NEW SEASON! \xF0...,2,positive
6,7,him @CHRISPRATFANFOREVER45 is soo funny !!! I ...,2,positive
7,8,All language films are good. I like exploring ...,2,positive
8,9,this @13Reasonswhy shouldnâ€™t end so soon. IM...,2,positive
9,10,I need a break from all these social media mes...,0,negative


In [37]:
#Download data after model prediction 
data.to_csv(r'SentimentAnalysed_data.csv', index = False)